In [1]:
import polars as pl
import pandas as pd
import numpy as np
import math as math

In [2]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

In [3]:
# normalize data
master_players = players.with_columns([pl.col('nflId').cast(str)])
plays = plays.join(games,on='gameId')
plays = plays.with_columns([
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId')
])

tracking = tracking.with_columns(
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId'),
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str) + '-'
     + pl.col('nflId').cast(str)).alias('uniquePlayerId'),
)

# normalize position
tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(53.3-pl.col('y')).otherwise(pl.col('y')).alias('adjustedX'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('x')).otherwise(120-pl.col('x')).alias('adjustedY')
])

tracking=tracking.with_columns([
    pl.when(pl.col('event').is_in(['ball_snap','autoevent_ballsnap'])).then(pl.col('frameId')).otherwise(-1)
    .alias('startingFrameId'),
])
tracking=tracking.with_columns([
    pl.col('startingFrameId').max().over(pl.col('uniquePlayId')),
])
tracking=tracking.with_columns([
    (pl.col('frameId') - pl.col('startingFrameId')).alias('framesSinceSnap'),
])
tracking = tracking.filter(pl.col('startingFrameId')!=-1)

# normalize orientation 'o' and direction 'dir'
# convert 'NA' to 0
replacement_values = {'NA': '0'}
tracking = tracking.with_columns(
    pl.col('o').apply(lambda x: replacement_values.get(x, x)),
    pl.col('dir').apply(lambda x: replacement_values.get(x, x)),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(pl.col('dir').cast(pl.Float64)).otherwise(180-pl.col('dir').cast(pl.Float64)).alias('adjustedDir'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('o').cast(pl.Float64)).otherwise((180+pl.col('o').cast(pl.Float64))%360).alias('firstAdjustedO'),
])

tracking=tracking.with_columns([
    pl.when(pl.col('firstAdjustedO') <= 180).then(180-pl.col('firstAdjustedO')).otherwise(540-pl.col('firstAdjustedO')).alias('adjustedO')
])

In [4]:
CONE_ANGLE_FAR = 50 # degrees
CONE_ANGLE_CLOSE = 120 # degrees
MAX_DISTANCE = 3 # yards
BLOCKING_RADIUS = 1.25 # yards

def angle_in_range(angle, left_boundary, right_boundary):
    return (angle - left_boundary) % 360 <= (right_boundary - left_boundary) % 360

# row = [o, dir, adjustedX, adjustedY, oDefender, dirDefender, adjustedXDefender, adjustedYDefender]
def looking_to_block_or_blocking_df_fn(row) -> int:
    blocking_status = 0
    player1 = row[0:4]
    player2 = row[4:]

    if is_in_vision_cone(player1, player2, CONE_ANGLE_FAR, MAX_DISTANCE) or is_in_vision_cone(player1, player2, CONE_ANGLE_CLOSE, BLOCKING_RADIUS):
        blocking_status = 1

    if is_blocking(player1, player2) and blocking_status == 1:
        blocking_status = 2

    return blocking_status
    

def looking_to_block_or_blocking(player1: tuple, player2: tuple) -> int:
    blocking_status = 0
    if is_in_vision_cone(player1, player2):
        blocking_status = 1

    if is_blocking(player1, player2) and blocking_status == 1:
        blocking_status = 2

    return 0

def is_in_vision_cone(player1: tuple, player2: tuple, cone_angle, distance) -> bool:
    return (is_in_angle(player1, player2, cone_angle) and is_in_distance(player1, player2, distance))

def is_in_angle(player1: tuple, player2: tuple, cone_angle) -> bool:
    half_cone_angle = cone_angle / 2

    y_dist = player2[3] - player1[3]
    x_dist = player2[2] - player1[2]
    player1_orientation = float(player1[0]) if type(player1[0]) == str else player1[0]

    # print("isinangle")
    # print("player1")
    # print(player1)

    # print("player2")
    # print(player2)

    # print("player1_ori")
    # print(player1_orientation)

    angle = math.degrees(math.atan2(y_dist, x_dist))
    angle = (angle + 360) % 360
    
    left_vision_boundary = (360 + player1_orientation - half_cone_angle) % 360
    right_vision_boundary = (player1_orientation + half_cone_angle) % 360
    
    
#     print(left_vision_boundary)
#     print(angle)
#     print(right_vision_boundary)
#     print(angle_in_range(angle,left_vision_boundary,right_vision_boundary))

    # print("angle")
    # print(angle)

    if angle_in_range(angle,left_vision_boundary,right_vision_boundary):
        return True
    
    return False

def is_in_distance(player1: tuple, player2: tuple, dist=MAX_DISTANCE) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    if distance_between_players <= dist:
        return True
    
    return False

def is_blocking(player1: tuple, player2: tuple) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    if distance_between_players <= BLOCKING_RADIUS:
        return True

    return False


def calculate_distance(player1: tuple, player2: tuple) -> float:
    y_dist = abs(player1[3] - player2[3])
    x_dist = abs(player1[2] - player2[2])

    # print(y_dist)
    # print(x_dist)
    # print(math.sqrt(x_dist**2 + y_dist**2))

    return math.sqrt(x_dist**2 + y_dist**2)

In [58]:
# We need copy so we don't modify the original list :)
import copy
DENT_CONE_ANGLE = 30
DENT_DISTANCE = 1.25
SIDE_CONE_ANGLES = 180

# 0 => None
# 1 => Box -> Left
# 2 => Dent
# 3 => Spill -> Right
def type_of_block_df_fn(row) -> str:    
    player1 = list(row[0:4])
    player2 = list(row[4:])
    
    if is_dent(player1, copy.copy(player2)):
        return "headup"
    if is_left_vision_cone(player1, copy.copy(player2)):
        return "left"
    elif is_right_vision_cone(player1, copy.copy(player2)):
        return "right"
    
    return "none"

def is_left_vision_cone(player1: tuple, player2: tuple) -> bool:
    player2[0] = (player2[0] + 90) % 360
    return is_in_angle(player2, player1, SIDE_CONE_ANGLES)

# Both players can see each other
# Only need to check player2 vision cone since we calculate blockType based on player1 earlier
def is_dent(player1: tuple, player2: tuple) -> bool:
    return is_in_vision_cone(player2, player1, DENT_CONE_ANGLE, DENT_DISTANCE)

def is_right_vision_cone(player1: tuple, player2: tuple) -> bool:
    player2[0] = (player2[0] - 90) % 360
    return is_in_angle(player2, player1, SIDE_CONE_ANGLES)
    

In [66]:
GAME_ID = 2022091107
PLAY_ID = 254
JERSEY_NUMBER = '83'
JERSEY_NUMBER_DEFENDER = '90'


labeled = tracking.filter(pl.col('gameId')==GAME_ID).filter(pl.col('playId')==PLAY_ID)
players = master_players

players = labeled.join(players,on='nflId',how='left')
players = players.filter(pl.col('club')!='football')
players = players.join(games.select(['gameId','homeTeamAbbr','visitorTeamAbbr']),on='gameId')
players = players.with_columns([
    pl.when(pl.col('club')==pl.col('homeTeamAbbr'))
    .then(pl.col('visitorTeamAbbr'))
    .otherwise(pl.col('homeTeamAbbr'))
    .alias('opponentClub')
])

In [67]:
players = players.join(
    players,
    left_on=['gameId','playId','frameId','club'],
    right_on=['gameId','playId','frameId','opponentClub'],
    suffix='Defender'
)

In [68]:
blocking_df = players.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender'
).apply(looking_to_block_or_blocking_df_fn)

blocking_df = blocking_df.to_series()
players = players.with_columns(blockType=blocking_df)

In [69]:
test_data = players.filter(pl.col('gameId')==GAME_ID).filter(pl.col('playId')==PLAY_ID)
# test_data = test_data.filter(pl.col('jerseyNumber')=='73').filter(pl.col('framesSinceSnap')==12)
# test_data = test_data.filter(pl.col('jerseyNumberDefender')=='91')

is_blocking_df = test_data.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 
    'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender'
).apply(looking_to_block_or_blocking_df_fn)

is_blocking_df = is_blocking_df.to_series()
test_data = test_data.with_columns(blockType=is_blocking_df)

In [70]:
type_of_block_df = test_data.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 
    'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender', 'blockType'
).apply(type_of_block_df_fn)

test_data = test_data.with_columns(blockLocation=type_of_block_df.to_series())

In [72]:
test_blocking = test_data.filter(pl.col('jerseyNumber')=='99')#.filter(pl.col('jerseyNumberDefender')==JERSEY_NUMBER_DEFENDER)
test_blocking.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 
    'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender', 
    'jerseyNumber','jerseyNumberDefender','blockType',
    'framesSinceSnap', 'blockLocation'
).filter(pl.col('framesSinceSnap')>=5).filter(pl.col('blockType')>0)\
.sort(by='framesSinceSnap').head(20).tail(20)


adjustedO,adjustedDir,adjustedX,adjustedY,adjustedODefender,adjustedDirDefender,adjustedXDefender,adjustedYDefender,jerseyNumber,jerseyNumberDefender,blockType,framesSinceSnap,blockLocation
f64,f64,f64,f64,f64,f64,f64,f64,str,str,i64,i64,str
51.98,226.91,23.68,44.6,262.39,298.36,27.82,46.14,"""64""","""91""",0,5,"""right"""
57.55,216.73,23.82,44.5,264.73,304.99,27.77,46.08,"""64""","""91""",0,6,"""right"""
58.75,208.06,23.98,44.4,261.63,309.58,27.7,46.0,"""64""","""91""",0,7,"""right"""
58.75,199.71,24.16,44.33,260.41,312.27,27.6,45.89,"""64""","""91""",0,8,"""right"""
58.75,194.47,24.35,44.27,263.6,313.82,27.49,45.78,"""64""","""91""",0,9,"""right"""
60.93,181.52,24.59,44.27,257.38,318.61,27.33,45.65,"""64""","""91""",0,10,"""right"""
62.4,176.35,24.81,44.27,254.19,320.32,27.17,45.52,"""64""","""91""",0,11,"""right"""
66.69,170.85,25.03,44.3,246.17,327.76,26.97,45.43,"""64""","""91""",0,12,"""right"""
71.68,165.75,25.22,44.34,233.15,332.3,26.78,45.35,"""64""","""91""",0,13,"""right"""


In [55]:
# take third frame after contact to check dent vs spill